In [129]:
import fxcmpy
TOKEN = "65aa373e2c6d310060a655ac665b46531ecd1adc"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

|ERROR|2022-01-14 21:10:55,387|Socket returns unknown error.


ServerError: Can not connect to FXCM Server.

In [127]:
import datetime as dt
from icecream import ic 


start = dt.datetime(2022,1,12)
end = dt.datetime(2022,1,15)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end
data

|ERROR|2022-01-14 21:10:48,629|Socket returns unknown error.
|ERROR|2022-01-14 21:10:48,631|Connection aborted, failed to reconnect


OSError: Connection aborted, failed to reconnect

In [2]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])

In [3]:
from icecream import ic
from datetime import datetime

# d1 = pd.read_csv("dec1.csv")
# d2 = pd.read_csv("dec3.csv")
d3 = pd.read_csv("till_15jan.csv")
frames = [d3]
data = pd.concat(frames)
data.index = data.date.values

# # data = data.loc[(data["date"] > datetime.strptime("2022-01-05", "%Y-%m-%d"))]
data = data.drop(["date"], axis=1)
# data = data.iloc[40:3000] # 5831
data
# pd.DataFrame(index=data.date, data=data)

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
2022-01-10 00:00:00,1.13530,1.13524,1.13531,1.13509,1.13542,1.13535,1.13543,1.13520,91
2022-01-10 00:01:00,1.13523,1.13524,1.13524,1.13518,1.13535,1.13535,1.13535,1.13530,49
2022-01-10 00:02:00,1.13523,1.13529,1.13529,1.13523,1.13535,1.13541,1.13541,1.13535,32
2022-01-10 00:03:00,1.13528,1.13531,1.13534,1.13528,1.13541,1.13543,1.13547,1.13541,40
2022-01-10 00:04:00,1.13531,1.13523,1.13532,1.13522,1.13544,1.13534,1.13545,1.13533,62
...,...,...,...,...,...,...,...,...,...
2022-01-14 14:38:00,1.14416,1.14403,1.14425,1.14378,1.14429,1.14414,1.14438,1.14390,546
2022-01-14 14:39:00,1.14403,1.14392,1.14406,1.14383,1.14415,1.14406,1.14417,1.14395,373
2022-01-14 14:40:00,1.14393,1.14419,1.14426,1.14393,1.14406,1.14431,1.14438,1.14405,507
2022-01-14 14:41:00,1.14418,1.14414,1.14439,1.14404,1.14431,1.14425,1.14451,1.14416,461


In [6]:
# import ta
# from ta.trend import *
# from ta.momentum import *
from IPython.display import display


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)

# data

In [4]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *

class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# # for i in range(51,100):
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# df = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]


# df = df.loc[(df["STOCH_SIGNAL_F"] == "CHUTIYA" )]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["SMALL_SMA"] <= df.bidopen) & (df["SMALL_SMA"] <= df.bidclose) &
#            (df["LARGE_SMA"] <= df.bidopen) & (df["LARGE_SMA"] <= df.bidclose)) | 
#            ((df["SMA_SIGNAL"] == "SELL") & (df["SMALL_SMA"] >= df.bidopen) & (df["SMALL_SMA"] >= df.bidclose) &
#            (df["LARGE_SMA"] >= df.bidopen) & (df["LARGE_SMA"] >= df.bidclose))
#            ]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["RSI"] >= 70)) | ((df["SMA_SIGNAL"] == "SELL") & (df["RSI"] <= 30))]
# df = df.loc[:, ["bidopen","bidclose","SMA_SIGNAL", "SMALL_SMA", "LARGE_SMA", "RSI", "STOCH","STOCH_SIGNAL"]]
# df.drop(["askopen","askclose", "askhigh", "tickqty"], axis=1)
# df
# df.insert(loc = 0,column = 'Strategy Valid?',value = '<input type="checkbox" />')
# df.tail(50)
# df.style
# df

In [1014]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [17]:
class SMAStrategy2(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER" and \
                    last_sma_crossover.min_bid > last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and \
                    last_sma_crossover.max_bid < last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).iloc[1650:1700]

In [23]:
class SMAStrategy3(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         last_sma_crossover.min_bid > last_sma_crossover.small_sma:
            #         data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         break
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    
            #         last_sma_crossover.max_bid < last_sma_crossover.small_sma:
            #         data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy3()
# strategy.dataframe = data[:200]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
sig = {'bidopen': {'2022-01-10 03:37:00': 1.13342},
 'bidclose': {'2022-01-10 03:37:00': 1.13353},
 'bidhigh': {'2022-01-10 03:37:00': 1.13357},
 'bidlow': {'2022-01-10 03:37:00': 1.13341},
 'askopen': {'2022-01-10 03:37:00': 1.13353},
 'askclose': {'2022-01-10 03:37:00': 1.13365},
 'askhigh': {'2022-01-10 03:37:00': 1.13369},
 'asklow': {'2022-01-10 03:37:00': 1.13353},
 'tickqty': {'2022-01-10 03:37:00': 49},
 'max_bid': {'2022-01-10 03:37:00': 1.13357},
 'min_bid': {'2022-01-10 03:37:00': 1.13341},
 'small_sma': {'2022-01-10 03:37:00': 1.1335342857142856},
 'large_sma': {'2022-01-10 03:37:00': 1.133574},
 'rsi': {'2022-01-10 03:37:00': 40.06032299810913},
 'rsi_signal': {'2022-01-10 03:37:00': 'UP_REVERSE'},
 'sma_signal': {'2022-01-10 03:37:00': ''},
 'entry_price': {'2022-01-10 03:37:00': 1.13353},
 'entry_signal': {'2022-01-10 03:37:00': 'BUY'}}
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty","bidhigh", "bidlow", "asklow",
#         ], axis=1).head(50)
# dd

In [ ]:
class SMAStrategy3Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        signal = self.signal
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         last_sma_crossover.min_bid > last_sma_crossover.small_sma:
            #         data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         break
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    
            #         last_sma_crossover.max_bid < last_sma_crossover.small_sma:
            #         data = new_df[last_sma_crossover.name : last_sma_crossover.name]
            #         break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

In [178]:
class SMAStrategy2Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        # if signal and not data.empty:
        #     d_date = data.index[0]
        #     d = data.to_dict()
        #     d["entry_price"] = d["bidclose"]
        #     d["entry_signal"] = {d_date: signal}
        #     return d
        return df
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1)
# dd

In [24]:
test_data = data.copy()[:500]
import backtest
import strategy
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)


sma_backtest = BackTestStrategy(SMAStrategy3, test_data,target=0.0005,stoploss=0.0005, order_quantity=30000)
sma_backtest.start_backtesting()

'backtesting has been completed, please generate report'

In [25]:
import strategy
importlib.reload(strategy)
import backtest
importlib.reload(backtest)
kd = sma_backtest.get_report()
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
kd.head(36)
# 16 Total Stoploss
# 21 Total Target
# (38, 12)

running default exit strategy for the evaluation
5 Total Stoploss
6 Total Target
(11, 12)


,Index,bidopen,max_bid,min_bid,rsi,sma_signal,entry_price,entry_signal,exit_price,exit_time,status,pl
0,2022-01-10 00:45:00,1.13570,1.13570,1.13555,56.545446,,1.13558,SELL,1.13516,2022-01-10 01:48:00,TG,12.6
1,2022-01-10 02:25:00,1.13454,1.13467,1.13454,30.441067,,1.13466,BUY,1.13415,2022-01-10 02:53:00,SL,-15.3
2,2022-01-10 03:12:00,1.13375,1.13375,1.13373,30.136146,,1.13375,BUY,1.13331,2022-01-10 03:36:00,SL,-13.2
3,2022-01-10 03:37:00,1.13342,1.13357,1.13341,40.060323,,1.13353,BUY,1.13398,2022-01-10 04:06:00,TG,13.5
4,2022-01-10 04:09:00,1.13397,1.13397,1.13385,63.622259,,1.13386,SELL,1.13346,2022-01-10 04:51:00,TG,12.0
5,2022-01-10 04:59:00,1.13327,1.13336,1.13327,37.261400,,1.13336,BUY,1.13295,2022-01-10 06:07:00,SL,-12.3
6,2022-01-10 06:08:00,1.13295,1.13303,1.13295,33.154378,,1.13303,BUY,1.13260,2022-01-10 06:27:00,SL,-12.9
7,2022-01-10 06:28:00,1.13262,1.13271,1.13258,32.176600,,1.13271,BUY,1.13313,2022-01-10 06:51:00,TG,12.6
8,2022-01-10 07:01:00,1.13331,1.13333,1.13307,52.607993,,1.13315,SELL,1.13357,2022-01-10 07:16:00,SL,-12.6
9,2022-01-10 08:14:00,1.13214,1.13233,1.13204,38.638782,,1.13232,BUY,1.13278,2022-01-10 08:21:00,TG,13.8


In [924]:
# rk = pd.DataFrame(lucha)
# print(len(list(filter(lambda k : k == "SL", rk.status))) , "Total Stoploss")
# print(len(list(filter(lambda k : k == "TG", rk.status))) , "Total Target")
# rk

In [175]:
dk = kd.head(36)
print(len(list(filter(lambda k : k == "SL", dk.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", dk.status))) , "Total Target")
dk.pl.sum()

17 Total Stoploss
19 Total Target


31.799999999975167